In [36]:
import pandas as pd
import pickle
import numpy as np

class DataExtractor:

    type_conv = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'}
    
    def __init__(self, pkl_path, expired_path):
        self.pkl_path = pkl_path
        self.expired_path = expired_path
        
        
    def load_file(self):
        with open(self.pkl_path, 'rb') as file:
            data_pkl = pickle.load(file)
        return pd.DataFrame(data_pkl)
        

    def get_expired(self):
        with open (self.expired_path, 'r') as exp:
            return  exp.readlines()
    
    def flatten(self, row):
        flat = []
        total_invoice = 0
        for item in row["items"]:
            total_invoice +=item['item']['unit_price'] * int(item['quantity'])
            
        for item in row["items"]:   
            invoice_item = item['item'] 
            total_price = invoice_item['unit_price'] * item['quantity']
            percentage_in_invoice = total_price / total_invoice
            flattened_invoice = {
                'invoice_id': row['id'],
                'created_on': pd.to_datetime(row['created_on']),
                'invoiceitem_id': invoice_item['id'],
                'invoiceitem_name': invoice_item['name'],
                'type': self.type_conv[invoice_item['type']],
                'unit_price': invoice_item['unit_price'],
                'total_price':total_price,
                'percentage_in_invoice':percentage_in_invoice ,
                'is_expired': bool(row['id'] in self.get_expired())
            }
            flat.append(flattened_invoice)
        return flat

    def get_extracted_data(self):
        extracted_data = []
        df_raw = self.load_file()
        for col, row in df_raw.iterrows():
            extracted_data.append(self.flatten(row))
        return pd.DataFrame(data_extracted)
            


    